# A Deep Dive into the S&P 500: Predicting Stock Prices
Kanishk Chinnapapannagari, Aarav Naveen, Avyay Potarlanka, and Melvin Rajendran

## Introduction

In [40]:
import pandas as pd
import os
import csv

## Data Collection

In [41]:
# All historical stock data for each ticker in one dataframe
data = pd.DataFrame()

In [42]:
# File path for folder with all .csv files
folder_path = 'sp500-data'

# Look at all files in folder
for file_name in os.listdir(folder_path):
    # Only process .csv files
    if file_name.endswith('.csv'):
        # Get full path for the current .csv file
        file_path = os.path.join(folder_path, file_name)
        # Create a temporary DataFrame
        temp = pd.read_csv(file_path)
        # Get ticker name
        ticker = file_name[0:-4]
        # Add ticker name as a column
        temp['Ticker'] = [ticker] * len(temp)
        # Combine to accumulator dataframe that stores all data
        data = pd.concat([data, temp])
        

In [43]:
# Reorder columns so that the ticker name is at the start
data = data.reindex(columns=['Ticker', 'Date', 'Low', 'Open', 'Volume', 'High', 'Close', "Adjusted Close"])

# Display data
data.head()

,Ticker,Date,Low,Open,Volume,High,Close,Adjusted Close
0,CSCO,16-02-1990,0.073785,0.0,940636800.0,0.079861,0.077257,0.054862
1,CSCO,20-02-1990,0.074653,0.0,151862400.0,0.079861,0.079861,0.056712
2,CSCO,21-02-1990,0.075521,0.0,70531200.0,0.078993,0.078125,0.055479
3,CSCO,22-02-1990,0.078993,0.0,45216000.0,0.081597,0.078993,0.056095
4,CSCO,23-02-1990,0.078125,0.0,44697600.0,0.079861,0.078559,0.055787


## Data Processing

## Exploratory Data Analysis and Data Visualization

## Data Analysis, Hypothesis Testing, and Machine Learning

## Insights and Policy Decision